# NP-Hard Problems

In [1]:
import networkx as nx
import pulp

We will use two graph datasets for this assignment, both available in [GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. The first is a well-known graph of the social network of a karate club [1]. The second is a network representing the western states power grid in the US [2].

In [2]:
karate = nx.read_gml('karate.gml')
power = nx.read_gml('power.gml')

[1] W. Zachary, An information flow model for conflict and fission in small groups, Journal of Anthropological Research 33, 452-473 (1977). 

[2] D. J. Watts and S. H. Strogatz, Collective dynamics of 'small world' networks, Nature 393, 440-442 (1998). 

## Independent Set

Recall that an *independent set* in a graph is a set of nodes such that no two nodes are adjacent (connected by an edge). Finding the maximum size of an independent set in a graph is an NP-hard problem, so there is no known polynomial-time algorithm to solve it.

Provide an *integer program* formulation for this problem. This should look identical to a linear program formulation, except you will restrict the variables corresponding to the nodes to be integer instead of fractional.

The graph is denoted $G = (V, E)$. For this problem, you should use a binary variable $x_i \in \{0,1\}$ for each node $i \in N$, and you shouldn't need any other variables.

### Solution:


#### ILP Goal: 
Maximize 
$\sum x_i$

#### Subject to:
$x_i \in \{0,1\} \ \forall \ 1 \leq i \leq n   \\   
x_i + x_j \leq 1\ \forall \ \{i,\ j\} \in E $

### Integer Programming for Graph

In [3]:
def independent_set_ip(graph):
    """Computes a maximum independent set of a graph using an integer program.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, int)]) The IP solution as a list of node-value pairs.
        
    """    
    # TODO: implement function
    problem = pulp.LpProblem("Independent set problem", pulp.LpMaximize)
    x = pulp.LpVariable.dict("x", [str(i) for i in graph.nodes()], 0, cat = pulp.LpInteger)
    
    for i in graph.nodes():
        problem += x[str(i)] <= 1
    
    for e in graph.edges():
        problem += x[str(e[0])] + x[str(e[1])] <= 1
    
    problem += pulp.lpSum([x[str(i)] for i in graph.nodes()])
    status = problem.solve()
    
    solution = []
    for i in graph.nodes():
        pairs = (int(i), int(x[str(i)].value()))
        solution.append(pairs)
    
    return solution
    

In [4]:
def set_weight(solution):
    """Computes the total weight of the solution of an LP or IP for independent set.
    
    Args:
      - solution (list[int, float]): the LP or IP solution
    
    Returns:
      (float) Total weight of the solution
    
    """
    return sum(value for (node, value) in solution)

In [5]:
karate_ind_set = independent_set_ip(karate)
print "Size of karate set = ", set_weight(karate_ind_set)
power_ind_set = independent_set_ip(power)
print "Size of power set = ", set_weight(power_ind_set)

Size of karate set =  20
Size of power set =  2738


### Linear Programming 

In [6]:
def independent_set_lp(graph):
    """Computes the solution to the linear programming relaxation for the
    maximum independent set problem.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, float)]) The LP solution as a list of node-value pairs.
        
    """    
    # TODO: implement function
    problem = pulp.LpProblem("Independent set problem", pulp.LpMaximize)
    x = pulp.LpVariable.dict("x", [str(i) for i in graph.nodes()], 0)
    
    for i in graph.nodes():
        problem += x[str(i)] <= 1
    
    for e in graph.edges():
        problem += x[str(e[0])] + x[str(e[1])] <= 1
        
    problem += pulp.lpSum([x[str(i)] for i in graph.nodes()])
    status = problem.solve()
    
    solution = []
    for i in graph.nodes():
        pairs = (int(i), float(x[str(i)].value()))
        solution.append(pairs)
        
    return solution    

Let's see how the LP solutions compare to those of the IP.

In [7]:
karate_ind_set_relax = independent_set_lp(karate)
print "Value of karate set = ", set_weight(karate_ind_set_relax)
power_ind_set_relax = independent_set_lp(power)
print "Value of power set = ", set_weight(power_ind_set_relax)

Value of karate set =  20.5
Value of power set =  2758.0


A heuristic way to convert a fractional solution to an independent set is as follows. For each node $i$, include the node in the set if $x_i > 1/2$, and discard it otherwise. This will yield a set of $a$ nodes which have $b$ edges between them. By removing at most one node for each edge, this yields an independent set of size at least $a - b$.

Implement this rounding procedure.

In [8]:
def round_solution(solution, graph):
    """Finds the subgraph corresponding to the rounding of
    a solution to the independent set LP relaxation.
    
    Args:
      - solution (list[(int, float)]): LP solution
      - graph (nx.Graph): the original graph
      
    Returns:
        (nx.Graph) The subgraph corresponding to rounded solution
    
    """
    # TODO: implement function
    sub = nx.Graph()\
    
    for e in graph.edges():
        sub.add_edge(e[0], e[1])
        
    for n in sub.nodes():
        index = [i[0] for i in solution].index(n)
        if solution[index][1] <= 1./2:
            sub.remove_node(n)
    return sub

The following function assesses the quality of the heuristic approach.

In [9]:
def solution_quality(rounded, optimal):
    """Computes the percent optimality of the rounded solution.
    
    Args:
      - rounded (nx.Graph): the graph obtained from rounded LP solution
      - optimal: size of maximum independent set
    
    """
    num_nodes = rounded.number_of_nodes() - rounded.number_of_edges()
    return float(num_nodes) / optimal

Let's check the quality of this approach compared to the optimal IP solutions.

In [10]:
karate_rounded = round_solution(karate_ind_set_relax, karate)
karate_quality = solution_quality(karate_rounded, set_weight(karate_ind_set))
print "Quality of karate rounded solution = {:.0f}%".format(karate_quality*100)

power_rounded = round_solution(power_ind_set_relax, power)
power_quality = solution_quality(power_rounded, set_weight(power_ind_set))
print "Quality of power rounded solution = {:.0f}%".format(power_quality*100)

Quality of karate rounded solution = 90%
Quality of power rounded solution = 95%
